# 4. Comparing Measured and True x and Q<sup>2</sup>

One of the main goals of analysis of simulated data is to determine how well we will be able to measure the quantities we set out to measure. Since we have access to 'truth' information as well as the 'observed' quantities for the same event, we can make simple comparison.

Let us start again from the $x$ and $Q^2$ calculations in a previous notebook.

## Importing packages

Depending on the versions of uproot and XRootD that you have installed, you may encouter a warning from uproot below. Nevertheless, because of the simple data format of the ATHENA ROOT files, we are able to ignore this warning.

In [ ]:
import numpy as np
import uproot as ur
import awkward as ak
import matplotlib.pyplot as plt

## Opening a file with uproot

To test uproot, we will open a sample file (a DIS simulation sample):

In [ ]:
server = 'root://sci-xrootd.jlab.org//osgpool/eic/'
file = 'ATHENA/RECO/deathvalley-v1.0/DIS/NC/18x275/minQ2=10/pythia8NCDIS_18x275_minQ2=10_beamEffects_xAngle=-0.025_hiDiv_vtxfix_4_084.0008.root'

In [ ]:
events = ur.open(server + file + ':events')

## Accessing the reconstructed particle momentum

For this analysis we will only use the three-momentum `p` and the particle identication code `pid`. We will select only electrons (`pid == 11`) and combine them with their initial momentum $\vec{p}_0$ which, in the ATHENA coordinate system, is in the negative $z$ direction by definition.

In [ ]:
reconstructed_particles = events['ReconstructedParticles'].arrays()

In [ ]:
kr1, kr2, kr3 = reconstructed_particles['ReconstructedParticles.p.x'], reconstructed_particles['ReconstructedParticles.p.y'], reconstructed_particles['ReconstructedParticles.p.z']
pidr = reconstructed_particles['ReconstructedParticles.pid']
mr = reconstructed_particles['ReconstructedParticles.mass']

Note that the last two events in this data file do not have any particles, so we truncate them to avoid issues down the road. This was also the case in the previous exercise but will lead to issues when matching the generated and reconstructed values.

In [ ]:
kr1

In [ ]:
kp1 = kr1[:-2]
kp2 = kr2[:-2]
kp3 = kr3[:-2]
pid = pidr[:-2]
m   = mr[:-2]

## Determining the momentum transfer $Q^2$

For all particles we can calculate the energy, which we will consider the zeroth component of the four-momentum $p$.

In [ ]:
kp0 = np.sqrt(m**2+(kp1**2+kp2**2+kp3**2))

The four-momentum of the incoming electron beam has only a $p_z$ and $E$ component.

In [ ]:
k3 = -18
m0 = 0.000511
k0 = np.sqrt(m0**2 + k3**2)

We can now calculate the components of the four-momentum transfer $q_\mu = (k_\mu - k'_\mu)$:

In [ ]:
q0 = k0 - kp0
q1 =    - kp1
q2 =    - kp2
q3 = k3 - kp3

With the four components we can form the squared four-momentum transfer, a scalar quantity, which is $Q^2 = -q^2$. Of course we also need to ensure our detected particle was an electron!

In [ ]:
is_electron = (pid == 11)
Q2_all = -(q0**2 - q1**2 - q2**2 - q3**2)[is_electron]

...but, there may be multiple electrons in the final state. We select those Q2 values that are the largest as the ones most likely to be correct.

In [ ]:
Q2 = ak.max(Q2_all, 1)

## Determining the momentum fraction $x$

In order to determine $x$ we also need the incoming proton momentum $\vec{p}$. While it might be appealing to think that the proton momentum must be exactly along the $z$ axis as well, this is not the case in the interaction points of the EIC. At interaction point 6 (IP6), the crossing angle is -25 mrad in the $xz$ plane. Thus, the proton four-momentum is:

In [ ]:
alpha = -0.025
p1 = 275 * np.sin(alpha)
p2 = 0
p3 = 275 * np.cos(alpha)
p0 = np.sqrt(0.938**2 + p1**2 + p2**2 + p3**2)

With this proton four-momentum we can now calculate the product $p \cdot q$, another scalar quantity:

In [ ]:
pq = (p0 * q0 - p1 * q1 - p2 * q2 - p3 * q3)[is_electron]

and finally also $x = \frac{Q^2}{2 pq}$:

In [ ]:
x_all = 0.5 * Q2 / pq

Same issue with x as for $Q^2$, we want to select the same kinematics corresponding to the largest value for $Q^2$

In [ ]:
x = ak.flatten(ak.to_numpy(x_all[Q2_all == ak.max(Q2_all, 1)]))

## Determining the true $x$ 

Because we have access to the `mcparticles` branch, we can determine the 'true' $x$. For the rest we follow exactly the same procedure as for the reconstructed particles. Note the same truncation as we did before.

In [ ]:
mcparts = events['mcparticles'].arrays()[:-2]
pdgID = mcparts['mcparticles.pdgID']
status = mcparts['mcparticles.genStatus']
pgen1,pgen2,pgen3 = mcparts['mcparticles.ps.x'], mcparts['mcparticles.ps.y'], mcparts['mcparticles.ps.z']
mgen = mcparts['mcparticles.mass']

In [ ]:
pgen0 = np.sqrt(mgen**2+(pgen1**2+pgen2**2+pgen3**2))

Both beam particles (with realistic accelerator beam effects), and detected electrons are stored. 
 * Beam particles have status `4`  
 * Final state particles have status `1`.
 * Electrons have PID code `11`
 * Protons have PID code `2212`
 
 Finally, we ensure we only calculate the scattered electron kinenatics for the highest energy electron in the event.

In [ ]:
is_beam_electron = np.logical_and(status==4, pdgID==11)
is_beam_proton = np.logical_and(status==4, pdgID==2212)
is_fs_electron = np.logical_and(status==1, pdgID==11)

kgen0 = pgen0[is_beam_electron]
kgen1 = pgen1[is_beam_electron]
kgen2 = pgen2[is_beam_electron]
kgen3 = pgen3[is_beam_electron]

Pgen0 = pgen0[is_beam_proton]
Pgen1 = pgen1[is_beam_proton]
Pgen2 = pgen2[is_beam_proton]
Pgen3 = pgen3[is_beam_proton]

kpgen0 = ak.max(pgen0[is_fs_electron], 1)
kpgen1 = ak.max(pgen1[is_fs_electron], 1)
kpgen2 = ak.max(pgen2[is_fs_electron], 1)
kpgen3 = ak.max(pgen3[is_fs_electron], 1)

Now we can calculate the true kinematic quantities as before.

In [ ]:
qgen0 = kgen0 - kpgen0
qgen1 = kgen1 - kpgen1
qgen2 = kgen2 - kpgen2
qgen3 = kgen3 - kpgen3
Q2gen = ak.flatten(-(qgen0**2 - qgen1**2 - qgen2**2 - qgen3**2))

In [ ]:
pqgen = ak.flatten(Pgen0 * qgen0 - Pgen1 * qgen1 - Pgen2 * qgen2 - Pgen3 * qgen3)
pkgen = ak.flatten(Pgen0 * kgen0 - Pgen1 * kgen1 - Pgen2 * kgen2 - Pgen3 * kgen3)
ygen = pqgen / pkgen

In [ ]:
xgen = 0.5 * Q2gen / pqgen

## Comparing the true $x$ and observed $x$

So, how well did we do? Let's take a look at the numbers first.

In [ ]:
print(x)
print(xgen)

In [ ]:
print(Q2)
print(Q2gen)

Not too bad! Let's make some plots.

In [ ]:
Q2_ratio = ak.to_numpy(Q2) / ak.to_numpy(Q2gen)

In [ ]:
plt.hist(Q2_ratio, range=[0.9, 1.1], bins = 50)
#plt.yscale('log')
plt.xlabel('meas $Q^2$ / true $Q^2$')
plt.ylabel('Number of events')
plt.show()

In [ ]:
Q2_bins = np.logspace(1,3,40)
plt.hist2d(ak.to_numpy(Q2gen), ak.to_numpy(Q2), bins = [Q2_bins, Q2_bins])
plt.xlabel('true $Q^2$ [GeV^2]')
plt.ylabel('meas $Q^2$ [GeV^2]')
plt.xscale('log')
plt.yscale('log')
plt.show()

Now we have to get the $x$ values to use the corresponding values.

In [ ]:
x_ratio = ak.to_numpy(x) / ak.to_numpy(xgen)

We can now plot both a histogram of the ratio and another scatter plot.

In [ ]:
plt.hist(x_ratio, range=[0.5, 1.5], bins = 40)
#plt.yscale('log')
plt.xlabel('meas $x$ / true $x$')
plt.ylabel('Number of events')
plt.show()

In [ ]:
x_bins = np.logspace(-4,0,40)
plt.hist2d(ak.to_numpy(xgen), ak.to_numpy(x), bins = [x_bins, x_bins])
plt.xscale('log')
plt.yscale('log')
plt.xlabel('true $x$ [GeV^2]')
plt.ylabel('meas $x$ [GeV^2]')
plt.show()

The resolution here looks quite a bit worse. In a realistic analysis, we would be using a combination of different reconstruction methods for the DIS kinematics based on where in phase space we are. This can dramatically improve the resolutions.